In [5]:
import numpy as np
import json
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

In [11]:
def chunkify(lst, n):
    """Split a list into chunks of size n."""
    return [lst[i:i + n] for i in range(0, len(lst), n)]

def duplicate(machine_texts, human_texts):
    # Step 1: Determine the number of duplications
    num_duplications = len(machine_texts) // len(human_texts)

    # Step 2: Duplicate the human texts
    expanded_human_texts = human_texts * num_duplications

    # Step 3: Sample the remaining number of human texts if needed
    remaining_texts_needed = len(machine_texts) - len(expanded_human_texts)
    expanded_human_texts += random.sample(human_texts, remaining_texts_needed)

    # Step 4: Merge and shuffle the lists
    print("human: " + str(len(expanded_human_texts)) + " mahcine: " + str(len(machine_texts)))
    balanced_texts = expanded_human_texts + machine_texts
    random.shuffle(balanced_texts)
    
    return balanced_texts


train_data_1 = []

with open('domain1_train.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        train_data_1.append(data)

train_data_2_human = []
train_data_2_machine = []
with open('domain2_train.json', 'r') as file:
    for line in file:
        data = json.loads(line)
        if data["label"] == 0: 
            train_data_2_human.append(data)
        else:
            train_data_2_machine.append(data)
# print(len())
print(len(train_data_2_human))
print(len(train_data_2_machine))
balaned_data_2 = duplicate(train_data_2_machine, train_data_2_human)
print(len(balaned_data_2))

train_data = train_data_1 + balaned_data_2

12750
2150
human: 2150 mahcine: 2150
4300


In [12]:
X = np.array([entry["text"] for entry in train_data])
y = np.array([entry["label"] for entry in train_data])

NameError: name 'train_data' is not defined

In [101]:
#split training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [102]:
# BOW n-grams
#vectorizer = HashingVectorizer(n_features=10000)
vectorizer = CountVectorizer(ngram_range=(1,2))

X_train_text = [' '.join(map(str, sample)) for sample in X_train]
X_test_text = [' '.join(map(str, sample)) for sample in X_test]

X_train_transformed = vectorizer.fit_transform(X_train_text)
X_test_transformed = vectorizer.transform(X_test_text)

In [103]:
logistic_model = LogisticRegression(
    penalty="l2",
    max_iter=1000,
    class_weight="balanced"
)

### 插入部分

In [108]:
param_grid = {'C': [0.001, 0.01, 10]}
# grid_search = GridSearchCV(logistic_model, param_grid, cv=5)

In [109]:
grid_search = GridSearchCV(logistic_model, param_grid, cv=10)

In [110]:
grid_search.fit(X_train_transformed, y_train)

GridSearchCV(cv=10,
             estimator=LogisticRegression(class_weight='balanced',
                                          max_iter=1000),
             param_grid={'C': [0.001, 0.01, 10]})

In [111]:
best_model = grid_search.best_estimator_
# best_model.fit(X_train_transformed, y_train)


In [112]:
y_pred = best_model.predict(X_test_transformed)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")
# kaggle结果76.8%

Test Accuracy: 0.9302564102564103


### 插入结束

In [20]:
# logistic_model.fit(X_train_transformed, y_train)

LogisticRegression()

In [33]:
y_pred = best_model.predict(X_test_transformed)

# output acc
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9302564102564103


In [69]:
with open('logistic_output.csv', 'w') as output_file:
    output_file.write('id,class\n')  

    # read in dataset
    with open('test_set.json', 'r') as file:
        for line in file:
            entry = json.loads(line)
            text = entry["text"]

            # word embedding
            X_test = vectorizer.transform([" ".join(map(str, text))])

            # use logistic model to predict
            prediction = best_model.predict(X_test)

            # output csv
            output_file.write(f"{entry['id']},{prediction[0]}\n")